In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from POT import POT

In [8]:
def split_pos_neg(p):
    p_pos_full = np.zeros_like(p)#, dtype=np.float16)
    p_neg_full = np.zeros_like(p)#, dtype=np.float16)

    p_pos_indices = np.where(p > 0)[0]
    p_neg_indices = np.where(p < 0)[0]

    p_pos_full[p_pos_indices] = p[p_pos_indices]
    p_neg_full[p_neg_indices] = p[p_neg_indices]

    p_neg_full = np.abs(p_neg_full)

    return p_pos_full, p_neg_full

In [9]:
# Load pressure and DOA data for locations 2 and 4
pressure_files = [r'C:\Users\r0821191\Documents\helixon\csv\P_1.csv', r'C:\Users\r0821191\Documents\helixon\csv\P_3.csv']
doa_files = [r'C:\Users\r0821191\Documents\helixon\csv\doa_1.csv', r'C:\Users\r0821191\Documents\helixon\csv\doa_3.csv']

p_w_pos = []
p_w_neg = []
doa_data = []

for i in range(len(pressure_files)):
    df_pressure = pd.read_csv(pressure_files[i], header=None).values.flatten()#.astype(np.float16)
    df_pressure = df_pressure[400:40000]#.astype(np.float16)
    print("PRESSURE SHAPE: ")
    print(df_pressure.shape)
    print(df_pressure[0] )
    print()


    df_doa = pd.read_csv(doa_files[i], header=None).values#.astype(np.float16)
    df_doa = df_doa[400:40000]#.astype(np.float16)
    print("DOA SHAPE: ")
    print(df_doa.shape)
    print(df_doa[0])
    print()

    combined_df = pd.DataFrame({'pressure': df_pressure, 'doa_x': df_doa[:,0], 'doa_y': df_doa[:,1], 'doa_z': df_doa[:,2]})

    # Drop rows with any NaN values and get corresponding indices
    combined_df = combined_df.dropna()

    # Separate pressure data back
    df_pressure = combined_df['pressure'].values
    df_doa = combined_df[['doa_x', 'doa_y', 'doa_z']].values

    # Splitting positive and negative pressure
    p_pos, p_neg = split_pos_neg(df_pressure)
    p_w_pos.append(p_pos)
    p_w_neg.append(p_neg)
    doa_data.append(df_doa)

doa_data[0] = doa_data[0][0:30000]
doa_data[1] = doa_data[1][0:30000]
p_w_pos[0] = p_w_pos[0][0:30000]
p_w_pos[1] = p_w_pos[1][0:30000]
p_w_neg[0] = p_w_neg[0][0:30000]
p_w_neg[1] = p_w_neg[1][0:30000]


sum_pos=[p_w_pos[0].sum(), p_w_pos[1].sum()]
sum_neg=[p_w_neg[0].sum(), p_w_neg[1].sum()]

#normalize
p_w_pos[0]=p_w_pos[0]/sum_pos[0]
p_w_pos[1]=p_w_pos[1]/sum_pos[1]
p_w_neg[0]=p_w_neg[0]/p_w_neg[0]
p_w_neg[1]=p_w_neg[1]/p_w_neg[1]





FileNotFoundError: [Errno 2] No such file or directory: '/Users/valeriolorenzoni/Documents/maria/helixon/csv/P_1.csv'

In [4]:
# Create point clouds for positive and negative pressures
PC2_pos = {'pos': doa_data[0].reshape(-1,3), 'mass': p_w_pos[0], 'n': len(doa_data[0])}
PC4_pos = {'pos': doa_data[1].reshape(-1,3), 'mass': p_w_pos[1], 'n': len(doa_data[1])}

PC2_neg = {'pos': doa_data[0].reshape(-1,3), 'mass': p_w_neg[0], 'n': len(doa_data[0])}
PC4_neg = {'pos': doa_data[1].reshape(-1,3), 'mass': p_w_neg[1], 'n': len(doa_data[1])}
print("CHECK IF DIMENSIONS ARE RIGHT: ")
print(PC2_neg['pos'].shape)
print(PC2_pos['mass'].shape)
print(PC2_pos['n'])
print()

CHECK IF DIMENSIONS ARE RIGHT: 
(30000, 3)
(30000,)
30000



In [5]:
# Set expected distance and tolerance
distEx = np.nanmean(np.abs(PC4_pos['pos'] - PC2_pos['pos']))#.astype(np.float16)  # IDK WHAT TO PUT SO I DID THIS
distTol = 0.2 #np.float16(0.2)
print("DIST EX: ")
print(distEx)
print()


DIST EX: 
73.40655305702766



In [6]:
pot_pos = POT(PC2_pos, PC4_pos, distEx, distTol)
print("REAACHEEEDDDDDDDDDDDDD end of pot pos")
print(np.mat(pot_pos.T).shape)

COST MATRIX SHAPE: 
(30000, 30000)
DUMMY COST: 215.54088126856874
mass transported: 13838.353977242032 index: 0
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 28216
Cost coarse at index 0
mass transported: 13979.561670887359 index: 1
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 28784
Cost coarse at index 1
mass transported: 14120.769364532685 index: 2
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 29465
Cost coarse at index 2
REAACHEEEDDDDDDDDDDDDD end of pot pos
(30000, 30000)


In [7]:
pot_neg = POT(PC2_neg, PC4_neg, distEx, distTol)
print("REAACHEEEDDDDDDDDDDDDD end of pot neg")

COST MATRIX SHAPE: 
(30000, 30000)
DUMMY COST: 215.54088126856874
mass transported: 13837.966129571974 index: 0
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 28023
Cost coarse at index 0
mass transported: 13979.169865588014 index: 1
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 28604
Cost coarse at index 1
mass transported: 14120.373601604055 index: 2
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 29187
Cost coarse at index 2
REAACHEEEDDDDDDDDDDDDD end of pot neg


## END OF POT. INTERPOLATION OF PC BEGINS HERE

In [8]:
##PROLLY BETTER INTERPOLATION
k=0.5
PCk_pos_better = pot_pos.interpolatePC(k)
PCk_neg_better = pot_neg.interpolatePC(k)

In [ ]:
#denormalize
PCk_pos_better['pressure']=PCk_pos_better['pressure']*(sum_pos[0]+sum_pos[1])/2
PCk_neg_better['pressure']=PCk_neg_better['pressure']*(sum_neg[0]+sum_neg[1])/2

## MAKE CSVs
This takes the format 
method_PC-Interpolated_fromwhat-towhat_pressure/doa_pos/neg.csv
e.g. pot_2_1-3_p_pos.csv, pot_2_1-3_doa_pos.csv

In [10]:
pd.DataFrame(PCk_pos_better['pressure']).to_csv('interp_csv/pot_2_1-3_p_pos_mu_0001.csv', header=False, index=False)
pd.DataFrame(PCk_pos_better['doa']).to_csv('interp_csv/pot_2_1-3_doa_pos_mu_0001.csv', header=False, index=False)

pd.DataFrame(PCk_neg_better['pressure']).to_csv('interp_csv/pot_2_1-3_p_neg_mu_0001.csv', header=False, index=False)
pd.DataFrame(PCk_neg_better['doa']).to_csv('interp_csv/pot_2_1-3_doa_neg_mu_0001.csv', header=False, index=False)

In [10]:
### combine pc
all_pressures = np.concatenate(PCk_pos['mass'], PCk_neg['mass'])
all_doas = np.concatenate(PCk_pos['pos'], PCk_neg['pos'])

TypeError: only integer scalar arrays can be converted to a scalar index

In [2]:
### RIR
c=243
timeshifts = np.linalg.norm(all_doas, axis=1)/c

time = np.linspace(0,1,100000)
ir = np.zeros_like(time)

for i in range (len(all_pressures)):
    index = int(np.searchsorted(time, timeshifts[i]))
    ir[index]+= all_pressures[i]


plt.figure(figsize=(8,6))
plt.plot(time, ir)
plt.title('Ímpulse Response')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

NameError: name 'all_doas' is not defined